## 項目1 使用するPythonのバージョンを知っておく

In [58]:
!python --version

Python 3.8.5


In [59]:
import sys
print(sys.version_info)
print(sys.version)

sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


## 項目2 PEP8スタイルガイドに従う

以下のスタイルガイドがある。

* 空白
* 名前付け
* 式と文
* インポート

## 項目3 bytesとstrの違い

In [60]:
a = b'h\x65llo'
print(list(a))
print(a)

[104, 101, 108, 108, 111]
b'hello'


In [61]:
a = 'a\u0300 propos'
print(list(a))
print(a)

['a', '̀', ' ', 'p', 'r', 'o', 'p', 'o', 's']
à propos


Unicodeサンドイッチ（strとbytesの違いで処理に誤りがおきるので変換する）

In [62]:
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value # str instance

print(repr(to_str(b'foo')))
print(repr(to_str('bar')))

'foo'
'bar'


In [63]:
def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode('utf-8')
    else:
        value = bytes_or_str
    return value # bytes instance

print(repr(to_bytes(b'foo')))
print(repr(to_bytes('bar')))

b'foo'
b'bar'


その他の注意

* strとbytesでは同じように扱えない
  * str、bytes同士は+演算子で結合できる、比較、同値チェックができる
    * str、bytes間ではできないので注意

In [64]:
print(b'one', b'two')
print('one' + 'two')

b'one' b'two'
onetwo


In [65]:
b'one' + 'two'

TypeError: can't concat str to bytes

In [ ]:
'one' + b'two'

TypeError: can only concatenate str (not "bytes") to str

In [ ]:
assert b'red' > b'blue'
assert 'red' > 'blue'

In [ ]:
assert 'red' > b'blue'

TypeError: '>' not supported between instances of 'str' and 'bytes'

In [ ]:
assert b'blue' < 'red'

TypeError: '<' not supported between instances of 'bytes' and 'str'

In [ ]:
print(b'foo' == 'foo')

False


In [ ]:
print(b'red %s' % b'blue')
print('red %s' % 'blue')

b'red blue'
red blue


In [ ]:
print(b'red %s' % 'blue')

TypeError: %b requires a bytes-like object, or an object that implements __bytes__, not 'str'

In [ ]:
# bytesの__repr__によって b'blue' がそのまま表示される
print('red %s' % b'blue')

red b'blue'


ファイル書き込み（バイナリ、文字列の出力）

In [ ]:
with open('./random.bin', 'w') as f:
    f.write(b'\xf1\xf2\xf3\xf4\xf5')

TypeError: write() argument must be str, not bytes

In [ ]:
with open('./random.bin', 'wb') as f:
    f.write(b'\xf1\xf2\xf3\xf4\xf5')

In [ ]:
with open('./random.bin', 'r') as f:
    data = f.read()

UnicodeDecodeError: 'cp932' codec can't decode byte 0xf5 in position 4: incomplete multibyte sequence

In [ ]:
with open('chapter01/random.bin', 'rb') as f:
    data = f.read()
assert data == b'\xf1\xf2\xf3\xf4\xf5'

In [ ]:
with open('chapter01/random.bin', 'r', encoding='cp1252') as f:
    data = f.read()

assert data == 'ñòóôõ'

## 項目4 Cスタイルフォーマット文字列とstr.formatは使わずf文字列で埋め込む

### %による方法

* Cスタイルに慣れていればそのまま使える
* フォーマットの型を揃える必要がある（順序が変わるとフォーマット側も変える必要がある）
* 辞書型で渡すことができるがコードが冗長になる

In [ ]:
a = 0b10111011
b = 0xc5f
print('Binary is %d, hex is %d' % (a,b))

Binary is 187, hex is 3167


In [ ]:
key = 'my_var'
value = 1.234
formatted = '%-10s = %.2f' % (key, value)
print(formatted)

my_var     = 1.23


In [ ]:
key = 'my_var'
value = 1.234
formatted = '%-10s = %.2f' % (value, key)
print(formatted)

TypeError: must be real number, not str

In [ ]:
key = 'my_var'
value = 1.234
formatted = '%.2f = %-10s' % (key, value)
print(formatted)

TypeError: must be real number, not str

In [ ]:
pantry = [
    ('avocados', 1.25),
    ('bananas', 2.5),
    ('cherries', 15),
]
for i, (item, count) in enumerate(pantry):
    print('#%d: %-10s = %.2f' % (i, item, count))

#0: avocados   = 1.25
#1: bananas    = 2.50
#2: cherries   = 15.00


In [ ]:
# 書き換える必要がある
for i, (item, count) in enumerate(pantry):
    print('#%d: %-10s = %d' % (i, item, round(count)))

#0: avocados   = 1
#1: bananas    = 2
#2: cherries   = 15


In [ ]:
# 問題：nameを2回記述する必要がある
template = '%s loves food. See %s cook.'
name = 'Max'
formatted = template % (name, name)
print(formatted)

Max loves food. See Max cook.


In [ ]:
# 問題：メソッドを使う場合さらに長くなる
name = 'brad'
formatted = template % (name.title(), name.title())
print(formatted)

Brad loves food. See Brad cook.


In [ ]:
# 解決：辞書型で渡すと交代もできる
key = 'my_var'
value = 1.234

old_way = '%-10s = %.2f' % (key, value)

new_way = '%(key)-10s = %(value).2f' % {
    'key': key, 'value': value
}

reordered = '%(key)-10s = %(value).2f' % {
    'value': value, 'key': key
}

assert old_way == new_way == reordered

In [ ]:
# 解決：辞書型だと2つ入れずに入力できる
name = 'Max'

template = '%s loves food. See %s cook.'
before = template % (name, name)

template = '%(name)s loves food. See %(name)s cook.'
after = template % {'name': name}

assert after == before

In [ ]:
# 問題：変数とフォーマットをいれると冗長になる
for i, (item, count) in enumerate(pantry):
    before = '#%d: %-10s = %d' % (
        i + 1,
        item.title(),
        round(count)
    )

    after = '#%(loop)d: %(item)-10s = %(count)d' % {
        'loop': i + 1,
        'item': item.title(),
        'count': round(count)
    }

    assert before == after

In [ ]:
soup = 'lentil'
formatted = 'Today\'s soup is %(soup)s.' % {'soup': soup}
print(formatted)

Today's soup is lentil.


In [ ]:
menu = {
    'soup': 'lenti',
    'oyster': 'kumamoto',
    'special': 'schnitzel',
}
template = ('Today\'s soup is %(soup)s, '
            'buy one get two %(oyster)s oysters,'
            'and ourt special entree is %(special)s.')
formatted = template % menu
print(formatted)

Today's soup is lenti, buy one get two kumamoto oysters,and ourt special entree is schnitzel.


### 組み込みのformatとstr.format

* %演算子と比べると型指定せずにプレースホルダが使える利点がある
    * 型指定がないので順序変更に強い
* それ以外については記法が違うもののメリット・デメリットに大きな差はない

In [ ]:
a = 1234.5678
formatted = format(a, ',.2f')
print(formatted)

b = 'my string'
formatted = format(b, '^20s')
print('*', formatted, '*')

1,234.57
*      my string       *


In [ ]:
key = 'my_var'
value = 1.234

formatted = '{} = {}'.format(key, value)
print(formatted)

my_var = 1.234


In [ ]:
formatted = '{:<10} = {:.2f}'.format(key, value)
print(formatted)

my_var     = 1.23


In [ ]:
# エスケープ
print('%.2f%%' % 12.5)
print('{} replaces {{}}'.format(1.23))

12.50%
1.23 replaces {}


In [ ]:
# プレースホルダに順序指定ができる
formatted = '{1} = {0}'.format(key, value)
print(formatted)

1.234 = my_var


In [ ]:
# プレースホルダに順序をいれることで重複を省略できる
formatted = '{0} loves food. See {0} cook.'.format(name)
print(formatted)

Max loves food. See Max cook.


In [ ]:
# 変更に弱い点は変わらない
for i, (item, count) in enumerate(pantry):
    old_style = '#%d: %-10s = %d' % (
        i + 1,
        item.title(),
        round(count),
    )

    new_style = '#{}: {:<10s} = {}'.format(
        i + 1,
        item.title(),
        round(count),
    )

    assert old_style == new_style


In [ ]:
# 辞書のキーと組み合わせることもできるaが、冗長性の解消にはならない
formatted = 'First letter is {menu[oyster][0]!r}'.format(menu=menu)
print(formatted)

First letter is 'k'


In [ ]:
# 辞書型の連携も冗長さは変わらない
menu = {
    'soup': 'lenti',
    'oyster': 'kumamoto',
    'special': 'schnitzel',
}
old_template = ('Today\'s soup is %(soup)s, '
            'buy one get two %(oyster)s oysters, '
            'and our special entree is %(special)s.')
old_formatted = old_template % menu

new_template = (
    'Today\'s soup is {soup}, '
    'buy one get two {oyster} oysters, '
    'and our special entree is {special}.'
)
new_formatted = new_template.format(**menu)

assert new_formatted == old_formatted

### フォーマット済み文字列

Python3.6以降で登場。

* formatや%と比較して重複がない

In [ ]:
key = 'my_var'
value = 1.234

formatted = f'{key} = {value}'
print(formatted)

my_var = 1.234


In [ ]:
formatted = f'{key!r:<10} = {value:.2f}'
print(formatted)

'my_var'   = 1.23


In [ ]:
f_string = f'{key:<10} = {value:.2f}'
c_tuple = '%-10s = %.2f' % (key, value)
str_args = '{:<10} = {:.2f}'.format(key, value)
str_kw = '{key:<10} = {value:.2f}'.format(key=key, value=value)
c_dict = '%(key)-10s = %(value).2f' % {'key': key, 'value': value}

assert c_tuple == c_dict == f_string
assert str_args == str_kw == f_string

In [ ]:
pantry = [
    ('a', 1.234),
    ('b', 1.567),
    ('c', 5.345),
]
for i, (item, count) in enumerate(pantry):
    old_style = '#%d: %-10s = %d' % (
        i + 1,
        item.title(),
        round(count),
    )

    new_style = '#{}: {:<10s} = {}'.format(
        i + 1,
        item.title(),
        round(count),
    )

    f_string = f'#{i+1}: {item.title():<10s} = {round(count)}'

    assert old_style == new_style == f_string

In [ ]:
for i, (item, count) in enumerate(pantry):
    print(f'#{i+1}: '
            f'{item.title():<10s} = '
            f'{round(count)}')


#1: A          = 1
#2: B          = 2
#3: C          = 5


In [ ]:
places = 3
number = 1.23456
print(f'My number is {number:.{places}f}')

My number is 1.235


## 項目5 複雑な式の代わりにヘルパー関数を書く

In [ ]:
from urllib.parse import parse_qs

my_values = parse_qs('red=5&blue=0&green=', keep_blank_values=True)
print(repr(my_values))

{'red': ['5'], 'blue': ['0'], 'green': ['']}


In [ ]:
print('Red:     ', my_values.get('red'))
print('Green:   ', my_values.get('green'))
print('Opacity: ', my_values.get('opacity'))

Red:      ['5']
Green:    ['']
Opacity:  None


In [ ]:
red = my_values.get('red', [''])[0] or 0
green = my_values.get('green', [''])[0] or 0
opacity = my_values.get('opacity', [''])[0] or 0
print(f'Red:     {red!r}')
print(f'Green:   {green!r}')
print(f'Opacity: {opacity!r}')

Red:     '5'
Green:   0
Opacity: 0


In [ ]:
red = int(my_values.get('red', [''])[0] or 0)

In [ ]:
red_str = my_values.get('red', [''])
red = int(red_str[0]) if red_str[0] else 0

In [ ]:
green_str = my_values.get('green', [''])
if green_str[0]:
    green = int(green_str[0])
else:
    green = 0

In [ ]:
def get_first_int(values, key, default=0):
    found = values.get(key, [''])
    if found[0]:
        return int(found[0])
    else:
        return default

In [ ]:
green = get_first_int(my_values, 'green')

## 項目6 インデックスではなく複数代入アンパックを使う

In [ ]:
snack_calories = {
    'chips': 140,
    'popcorn': 80,
    'nuts': 190,
}
items = tuple(snack_calories.items())
print(items)

(('chips', 140), ('popcorn', 80), ('nuts', 190))


In [ ]:
item = ('Peanut butter', 'Jelly')
first = item[0]
second = item[1]
print(first, 'and', second)

Peanut butter and Jelly


In [ ]:
pair = ('Chocolate', 'Peanut butter')
pair[0] = 'Honey'

TypeError: 'tuple' object does not support item assignment

In [ ]:
item = ('Peanut butter', 'Jelly')
first, second = item #Unpack
print(first, 'and', second)

Peanut butter and Jelly


In [ ]:
favorite_snacks = {
    'salty': ('pretzels', 100),
    'sweet': ('cookies', 180),
    'veggie': ('carrots', 20),
}

(
    (type1, (name1, cals1)),
    (type2, (name2, cals2)),
    (type3, (name3, cals3)),
) = favorite_snacks.items()

print(f'Favorite {type1} is {name1} with {cals1} calories')
print(f'Favorite {type2} is {name2} with {cals2} calories')
print(f'Favorite {type3} is {name3} with {cals3} calories')

Favorite salty is pretzels with 100 calories
Favorite sweet is cookies with 180 calories
Favorite veggie is carrots with 20 calories


In [ ]:
def bubble_sort(a):
    for _ in range(len(a)):
        for i in range(1, len(a)):
            if a[i] < a[i-1]:
                temp = a[i]
                a[i] = a[i-1]
                a[i-1] = temp
names = ['pretzels', 'carrots', 'arugula', 'bacon']
bubble_sort(names)
print(names)

['arugula', 'bacon', 'carrots', 'pretzels']


In [ ]:
def bubble_sort(a):
    for _ in range(len(a)):
        for i in range(1, len(a)):
            if a[i] < a[i-1]:
                a[i-1], a[i] = a[i], a[i-1] #Swap
names = ['pretzels', 'carrots', 'arugula', 'bacon']
bubble_sort(names)
print(names)

['arugula', 'bacon', 'carrots', 'pretzels']


In [ ]:
snacks = [('bacon', 350), ('donut', 240), ('muffin', 190)]
for i in range(len(snacks)):
    item = snacks[i]
    name = item[0]
    calories = item[1]
    print(f'#{i+1}: {name} has {calories} calories')

#1: bacon has 350 calories
#2: donut has 240 calories
#3: muffin has 190 calories


In [ ]:
for rank, (name, calories) in enumerate(snacks, 1):
    print(f'#{rank}: {name} has {calories} calories')

#1: bacon has 350 calories
#2: donut has 240 calories
#3: muffin has 190 calories


## 項目7 rangeではなくenumerateを使う

In [ ]:
from random import randint

random_bits = 0
for i in range(64):
    if randint(0, 1):
            random_bits |= 1 << i
print(bin(random_bits))

0b11101111111111110001110001011111100111000001100110101100010100


In [ ]:
flavor_list = ['vanilla', 'chocolate', 'pecan', 'strawberry']
for flavor in flavor_list:
    print(f'{flavor} is delicious')

vanilla is delicious
chocolate is delicious
pecan is delicious
strawberry is delicious


In [ ]:
for i in range(len(flavor_list)):
  flavor = flavor_list[i]
  print(f'{i+1}: {flavor}')

1: vanilla
2: chocolate
3: pecan
4: strawberry


In [ ]:
it = enumerate(flavor_list)
print(next(it))
print(next(it))

(0, 'vanilla')
(1, 'chocolate')


In [ ]:
for i, flavor in enumerate(flavor_list):
  print(f'{i + 1}: {flavor}')

1: vanilla
2: chocolate
3: pecan
4: strawberry


In [ ]:
for i, flavor in enumerate(flavor_list, 1):
  print(f'{i}: {flavor}')

1: vanilla
2: chocolate
3: pecan
4: strawberry


## 項目8 イテレータを並列に処理するにはzipを使う

In [ ]:
names =['Cecilia', 'Lise', 'Marie']
counts = [len(n) for n in names]
print(counts)

[7, 4, 5]


In [ ]:
longest_name = None
max_count = 0
for i in range(len(names)):
  count = counts[i]
  if count > max_count:
    longest_name = names[i]
    max_count = count
print(longest_name)

Cecilia


In [ ]:
for i, name in enumerate(names):
  count = counts[i]
  if count > max_count:
    longest_name = name
    max_count = count
print(longest_name)

Cecilia


In [ ]:
# zipによる表現
for name, count in zip(name, counts):
  if count > max_count:
    longest_name = name
    max_letters = count
print(longest_name)

Cecilia


In [ ]:
# zipの長さが違う場合の挙動→最短の入力の長さ分しかループしないのでRosalindは無視される
names.append('Rosalind')
for name, count in zip(names, counts):
  print(name)

Cecilia
Lise
Marie


In [ ]:
# zip_longestを使えば最長の入力の長さ分ループしてくれる(短い分はNoneで埋められる)
import itertools

for name, count in itertools.zip_longest(names, counts):
  print(f'{name}: {count}')

Cecilia: 7
Lise: 4
Marie: 5
Rosalind: None


## 項目9 forループとwhileループのあとのelseブロックは使わない
ややこしいので、使わないほうが吉
* 最後までループが終わると実行されbreakされると実行されなくなる
  * ifのelseやtryのelseと比較して直感的でない

In [ ]:
# breakされなかったらelseブロックが実行される
for i in range(3):
  print('Loop', i)
else:
  print('Else block!!')

Loop 0
Loop 1
Loop 2
Else block!!


In [ ]:
# 途中でbreakされるとelseブロックが実行されない
for i in range(3):
  print('Loop', i)
  if i == 1:
    break
else:
  print('Else block!!')

Loop 0
Loop 1


In [ ]:
# 一度もループがない場合もelseは実行される
for x in []:
    print('Never run')
else:
    print('For Else block!!')

For Else block!!


In [ ]:
# whileがFalseのときも実行される
while False:
    print('Never runs!')
else:
    print('While Else block!!!')

While Else block!!!


In [ ]:
# Elseの使いよう（互いに素（公約数がない）であるかチェックするプログラム）
# 互いに素であればbreakされないので、elseが実行される
a = 4
b = 9
for i in range(2, min(a,b) + 1):
    print('Testing', i)
    if a % i == 0 and b % i == 0:
        print('Not coprime')
        break
else:
    print('Coprime')

Testing 2
Testing 3
Testing 4
Coprime


In [ ]:
# 関数化してreturnすればいい
def coprime(a, b):
    for i in range(2, min(a,b) + 1):
        if a % i == 0 and b % i == 0:
            return False
    return True 

assert coprime(4, 9)
assert not coprime(3, 6)

In [ ]:
# または変数を用意してbreakすればよい、いずれにしてもElseを使う必要はない
def coprime2(a, b):
    is_coprime = True
    for i in range(2, min(a,b) + 1):
        if a % i == 0 and b % i == 0:
            is_coprime = False
            break
    return is_coprime

assert coprime(4, 9)
assert not coprime(3, 6)

## 項目10 代入式で繰り返しを防ぐ

* Python3.8からの新機能で繰り返しを防ぐ
  * ウォルラス演算子

In [ ]:
# フルーツの残量を示す変数
fresh_fruit = {
    'apple': 10,
    'banana': 8,
    'lemon': 5,
}

In [ ]:
# if文でしか使わないcountを必要以上のスコープで定義している
def make_lemonade(count):
    pass

def out_of_stock():
    pass

count = fresh_fruit.get('lemon', 0)
if count:
    make_lemonade(count)
else:
    out_of_stock()

In [ ]:
# 解決法
# これまで同じように書くと 
if count := fresh_fruit.get('lemon', 0):
    make_lemonade(count)
else:
    out_of_stock()

In [ ]:
# バナナをスライスし、スムージーを作る作業をコード化
def slice_bananas(count):
    pass

class OutOfBananas(Exception):
    pass

def make_smoothies(count):
    pass

pieces = 0
count = fresh_fruit.get('banana', 0)
if count >= 2:
    pieces = slice_bananas(count)

try:
    smoothies = make_smoothies(pieces)
except OutOfBananas:
    out_of_stock()

In [ ]:
# elseを使えばpiecesに必ず値が入るので、最初に記述しなくていい（追いやすい）
count = fresh_fruit.get('banana', 0)
if count >= 2:
    pieces = slice_bananas(count)
else:
    pieces = 0

try:
    smoothies = make_smoothies(pieces)
except OutOfBananas:
    out_of_stock()

In [ ]:
# 代入式でcountの冗長をへらす
pieces = 0
if (count := fresh_fruit.get('banana', 0)) >= 2:
    pieces = slice_bananas(count)

try:
    smoothies = make_smoothies(pieces)
except OutOfBananas:
    out_of_stock()


In [ ]:

# 代入式＋pieceをelseに移行
if (count := fresh_fruit.get('banana', 0)) >= 2:
    pieces = slice_bananas(count)
else:
    pieces = 0

try:
    smoothies = make_smoothies(pieces)
except OutOfBananas:
    out_of_stock()

In [ ]:
def make_cider(count):
    pass

# Switch文がないので、if elseを使う例（冗長になってしまう例）
count = fresh_fruit.get('banana', 0)
if count >= 2:
    pieces = slice_bananas(count)
    to_enjoy = make_smoothies(pieces)
else:
    count = fresh_fruit.get('apple', 0)
    if count >= 4:
        to_enjoy = make_cider(count)
    else:
        count = fresh_fruit.get('lemon', 0)
        if count:
            to_enjoy = make_lemonade(count)
        else:
            to_enjoy = 'Nothing'

In [ ]:
# walrus演算子ではswitchっぽく書けるしインデントも減らせる
# インデントを減らさないで直接ifの中に関数をいれることもできるだろうけど、countを使い回せないのでifの中で呼び直す必要がある
if (count := fresh_fruit.get('banana', 0)) >= 2:
    pieces = slice_bananas(count)
    to_enjoy = make_smoothies(pieces)
elif (count := fresh_fruit.get('apple', 0)) >= 4:
    to_enjoy = make_cider(count)
elif count := fresh_fruit.get('lemon', 0):
    to_enjoy = make_lemonade(count)
else:
    to_enjoy = 'Nothing'

In [ ]:
# do-whileの例（Pythonにはdo-whileがない）
# doに当たる部分をwhile前に実行する必要がある（この場合fresh_fruit = pick_fruit())
fruits = [
    {'apple': 10},
    {'banana': 8},
    {'lemon': 5},
]
def pick_fruit():
    if fruits:
        return fruits.pop()
    else:
        return  None

def make_juice(fruit, count):
    return []

bottles = []
fresh_fruit = pick_fruit()
while fresh_fruit:
    for fruit, count in fresh_fruit.items():
        batch = make_juice(fruit, count)
        bottles.extend(batch)
    fresh_fruit = pick_fruit()

In [ ]:
# 解決法１：breakで終わらせる方法（loop-and-a-half）
# fresh_fruit = pick_fruit()の重複は防げる
fruits = [
    {'apple': 10},
    {'banana': 8},
    {'lemon': 5},
]

bottles = []
while True:
    fresh_fruit = pick_fruit()
    if not fresh_fruit:
        break
    else:
        for fruit, count in fresh_fruit.items():
            batch = make_juice(fruit, count)
            bottles.extend(batch)

In [ ]:
# 解決法2：walrus演算子により条件文部分で計算する
fruits = [
    {'apple': 10},
    {'banana': 8},
    {'lemon': 5},
]
bottles = []
while fresh_fruit := pick_fruit():
    for fruit, count in fresh_fruit.items():
        batch = make_juice(fruit, count)
        bottles.extend(batch)